# Splitting intensity figures

This notebook contains code to make high-quality figures showing the splitting intensity measurements and best-fitting sinusoid (the splitting vector) made using the SplitRacer software (Reiss et al., 2017).

## To-do
- Improve the uncertainty bounding. Should really be calculated across a whole range of backazimuths to get a true bound
- Expose the y-limits OR come up with a scheme to set them automatically.

In [ ]:
# --- Import libraries and setup styles ---
import pathlib
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
    plt.style.use("figures")
except OSError:
    warnings.warn("You have not added the 'publication.mplstyle' stylesheet to your"
                  " matplotlib config - continuing with matplotlib defaults.")
mpl.rcParams["font.family"] = "Helvetica"

In [ ]:
# --- Utility functions ---
def plot_results_scatter(ax, df, clr, category):
    """Plot the splitting intensity measurements."""
    
    df = df[df["category"].str.contains(category)]

    ax.scatter(df["baz"].values, df["SplitInt"].values, marker="x",
               color=clr, label=f"{category.title()} results")

def create_si_figure(splits, si_fit, station):
    """Create the splitting intensity figure."""
    # Create figure
    fig, ax = plt.subplots(1, figsize=(15, 10))

    # --- Plot splitting intensity best fit ---
    # Unpack variables from file
    fast_direction, delay_time, fast_err, delay_err = si_fit.loc[0]

    # Create splitting intensity function and uncertainties
    x = np.arange(0, 360, 0.01)
    best_fit = delay_time * np.sin(np.pi * (x - fast_direction) / 90)
    fit_min1 = (delay_time - delay_err) * np.sin(np.pi * (x - (fast_direction - fast_err)) / 90)
    fit_max1 = (delay_time + delay_err) * np.sin(np.pi * (x - (fast_direction + fast_err)) / 90)
    fit_min2 = (delay_time - delay_err) * np.sin(np.pi * (x - (fast_direction + fast_err)) / 90)
    fit_max2 = (delay_time + delay_err) * np.sin(np.pi * (x - (fast_direction - fast_err)) / 90)

    fit_min, fit_max = [], []
    for vals in zip(fit_min1, fit_min2, fit_max1, fit_max2):
        fit_min.append(min(vals))
        fit_max.append(max(vals))

    # Plot
    plt.plot(x, best_fit, label="SVD", color="red", linewidth=1.5, linestyle="--")
    plt.fill_between(x, fit_min, fit_max, color="red", alpha=0.05)

    # --- Plot splitting intensity scatter ---
    for category, clr in zip(["null", "average", "good"], ["#dddddd", "#fdae61", "#2c7bb6"]):
        plot_results_scatter(ax, splits, clr, category)

    # --- Configure plot details ---
    ax.set_ylabel("Splitting intensity")
    ax.set_xlabel("Backazimuth, °")
    plt.xlim([0, 360])
    plt.ylim([-1, 1])
    xtickrange = np.arange(0, 361, 30)
    ax.set_xticks(xtickrange)
    ax.set_xticklabels(xtickrange)
    plt.legend(fontsize=16, loc=0)
    plt.title(f"{station} - \u03C6 = {fast_direction:3.0f} \u00B1 "
              f"{fast_err:3.0f}\u00B0; \u03B4t = {delay_time:3.2f} "
              f"\u00B1 {delay_err:3.2f} s")

    # --- Save figure ---
    plt.savefig(output_dir / f"{station}_splitting_intensity.pdf",
                dpi=400, bbox_inches="tight")

In [ ]:
# --- SET PATHS ---
results_path = pathlib.Path("/path/to/results")
output_dir = pathlib.Path("/path/to/output/figures")

output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# --- Plot for specific station ---
station = "KKM"
splits = pd.read_csv(results_path / station / "Single_Splitting/splitting_results.txt",
                     delim_whitespace=True)
si_fit = pd.read_csv(results_path / station / "Single_Splitting/SI_results.txt",
                     header=None, delim_whitespace=True)

create_si_figure(splits, si_fit, station)

In [ ]:
# --- Plot for all stations ---
stations = list(results_path.glob("*/"))
for station in stations:
    splits = pd.read_csv(station / "Single_Splitting/splitting_results.txt",
                         delim_whitespace=True)
    si_fit = pd.read_csv(station / "Single_Splitting/SI_results.txt",
                         header=None, delim_whitespace=True)
    
    station = station.name
    
    create_si_figure(splits, si_fit, station)